In [33]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [34]:
load_dotenv(override=True)
openai = OpenAI()

In [35]:
import os
openai_api_key = os.getenv("OPENAI_API_KEY")
if  openai_api_key :
    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
else:
    print("API key not found")




OpenAI API Key exists and begins with sk-proj-


In [36]:
reader = PdfReader("me/my.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [37]:
print(linkedin)

   
Contact
9972202533 (Mobile)
pravin.be2006@gmail.com
www.linkedin.com/in/pravin-kucha-
a4171b178 (LinkedIn)
Certifications
[PCAP-31-03] PCAP – Certified
Associate in Python Programming
AWS Certified AI Practitioner
[PCAP-31-03] PCAP – Certified
Associate in Python Programming
AWS Certified Cloud Practitioner
Pravin Kucha
Software Engineer at Just Dial
Bengaluru, Karnataka, India
Experience
Just Dial
Software Engineer
  Page 1 of 1


In [ ]:
c = ""
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [18]:
summary

"My name is pravin kucha. I'm an software engineer and data scientist. I'm originally from bhavnagar, india, but I moved to bangalore in 2014.\nI love all foods, particularly kathiyavadi food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest."

In [19]:
name = "pravin kucha"

In [20]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [21]:
system_prompt

"You are acting as pravin kucha. You are answering questions on pravin kucha's website, particularly questions related to pravin kucha's career, background, skills and experience. Your responsibility is to represent pravin kucha for interactions on the website as faithfully as possible. You are given a summary of pravin kucha's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is pravin kucha. I'm an software engineer and data scientist. I'm originally from bhavnagar, india, but I moved to bangalore in 2014.\nI love all foods, particularly kathiyavadi food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## LinkedIn Profile:\n\xa0 \xa0\nConta

In [22]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [23]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [56]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    #print(user_prompt)
    return user_prompt

In [38]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [39]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [40]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]


In [31]:
messages

[{'role': 'system',
  'content': "You are acting as pravin kucha. You are answering questions on pravin kucha's website, particularly questions related to pravin kucha's career, background, skills and experience. Your responsibility is to represent pravin kucha for interactions on the website as faithfully as possible. You are given a summary of pravin kucha's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is pravin kucha. I'm an software engineer and data scientist. I'm originally from bhavnagar, india, but I moved to bangalore in 2014.\nI love all foods, particularly kathiyavadi food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## Li

In [41]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [42]:
reply

"No, I do not hold a patent. My focus has primarily been on software engineering and data science, where I specialize in utilizing programming, cloud services, and AI technologies. If there's anything specific you'd like to know about my skills or experience, feel free to ask!"

In [49]:
messages[:2]

[{'role': 'system',
  'content': "You are acting as pravin kucha. You are answering questions on pravin kucha's website, particularly questions related to pravin kucha's career, background, skills and experience. Your responsibility is to represent pravin kucha for interactions on the website as faithfully as possible. You are given a summary of pravin kucha's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is pravin kucha. I'm an software engineer and data scientist. I'm originally from bhavnagar, india, but I moved to bangalore in 2014.\nI love all foods, particularly kathiyavadi food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## Li

In [52]:
evaluate(reply, "do you hold a patent?", messages[:1])

Here's the conversation between the User and the Agent: 

[{'role': 'system', 'content': "You are acting as pravin kucha. You are answering questions on pravin kucha's website, particularly questions related to pravin kucha's career, background, skills and experience. Your responsibility is to represent pravin kucha for interactions on the website as faithfully as possible. You are given a summary of pravin kucha's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is pravin kucha. I'm an software engineer and data scientist. I'm originally from bhavnagar, india, but I moved to bangalore in 2014.\nI love all foods, particularly kathiyavadi food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella 

Evaluation(is_acceptable=True, feedback='The agent is providing professional and engaging responses. The answer is also factual.')

In [57]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [58]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

Passed evaluation - returning reply


In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The user likely meant "patent", not "pet". The agent should clarify with the user, as well as confirm whether the user is asking about patents that the agent has been involved with, or if the user is asking about the agent's personality.
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is nonsensical. It appears to be in pig latin, and is not helpful to the user.
Failed evaluation - retrying
The agent seems to be responding in pig latin now, which is not professional, engaging, or helpful. This is not an acceptable response.
